# Classification in PySpark's MLlib Project Solution

### Genre classification
Now it's time to leverage what we learned in the lectures to a REAL classification project! Have you ever wondered what makes us, humans, able to tell apart two songs of different genres? How we do we inherenly know the difference between a pop song and heavy metal? This type of classifcation may seem easy for us, but it's a very difficult challenge for a computer to do. So the question is, could an automatic genre classifcation model be possible? 

For this project we will be classifying songs based on a number of characteristics into a set of 23 electronic genres. This technology could be used by an application like Pandora to recommend songs to users or just create meaningful channels. Super fun!

### Dataset
*beatsdataset.csv*
Each row is an electronic music song. The dataset contains 100 song for each genre among 23 electronic music genres, they were the top (100) songs of their genres on November 2016. The 71 columns are audio features extracted of a two random minutes sample of the file audio. These features have been extracted using pyAudioAnalysis (https://github.com/tyiannak/pyAudioAnalysis).

### Your task
Create an algorithm that classifies songs into the 23 genres provided. Test out several different models and select the highest performing one. Also play around with feature selection methods and finally try to make a recommendation to a user.  

For the feature selection aspect of this project, you may need to get a bit creative if you want to select features from a non-tree algorithm. I did not go over this aspect of PySpark intentionally in the previous lectures to give you chance to get used to researching the PySpark documentation page. Here is the link to the Feature Selectors section of the documentation that just might come in handy: https://spark.apache.org/docs/latest/ml-features.html#feature-selectors

Good luck! Have fun :)

### Source
https://www.kaggle.com/caparrini/beatsdataset

In [1]:
# First let's create our PySpark instance
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Classification").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

You are working with 1 core(s)


In [2]:
# Read in functions we will need
from pyspark.ml.feature import VectorAssembler #to combine all features
from pyspark.sql.types import * 
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer #convert labels to numeric if they are string
from pyspark.ml.feature import MinMaxScaler # for normalization
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorSlicer #for feature selection
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import Row

In [3]:
# To display ALL columns
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
path ="../Datasets/"
df = spark.read.csv(path+'beatsdataset.csv',inferSchema=True,header=True)
df.limit(5).toPandas()

,_c0,1-ZCRm,2-Energym,3-EnergyEntropym,4-SpectralCentroidm,5-SpectralSpreadm,6-SpectralEntropym,7-SpectralFluxm,8-SpectralRolloffm,9-MFCCs1m,10-MFCCs2m,11-MFCCs3m,12-MFCCs4m,13-MFCCs5m,14-MFCCs6m,15-MFCCs7m,16-MFCCs8m,17-MFCCs9m,18-MFCCs10m,19-MFCCs11m,20-MFCCs12m,21-MFCCs13m,22-ChromaVector1m,23-ChromaVector2m,24-ChromaVector3m,25-ChromaVector4m,26-ChromaVector5m,27-ChromaVector6m,28-ChromaVector7m,29-ChromaVector8m,30-ChromaVector9m,31-ChromaVector10m,32-ChromaVector11m,33-ChromaVector12m,34-ChromaDeviationm,35-ZCRstd,36-Energystd,37-EnergyEntropystd,38-SpectralCentroidstd,39-SpectralSpreadstd,40-SpectralEntropystd,41-SpectralFluxstd,42-SpectralRolloffstd,43-MFCCs1std,44-MFCCs2std,45-MFCCs3std,46-MFCCs4std,47-MFCCs5std,48-MFCCs6std,49-MFCCs7std,50-MFCCs8std,51-MFCCs9std,52-MFCCs10std,53-MFCCs11std,54-MFCCs12std,55-MFCCs13std,56-ChromaVector1std,57-ChromaVector2std,58-ChromaVector3std,59-ChromaVector4std,60-ChromaVector5std,61-ChromaVector6std,62-ChromaVector7std,63-ChromaVector8std,64-ChromaVector9std,65-ChromaVector10std,66-ChromaVector11std,67-ChromaVector12std,68-ChromaDeviationstd,69-BPM,70-BPMconf,71-BPMessentia,class
0,0,0.136440,0.088861,3.201201,0.262825,0.249212,1.114423,0.007003,0.256682,-22.723259,1.594074,0.011276,0.204468,0.042072,0.048552,0.158505,0.118984,-0.147956,-0.186152,-0.026418,-0.007264,-0.017900,0.011581,0.008747,0.041081,0.014497,0.025711,0.012587,0.060170,0.002864,0.004631,0.009576,0.026079,0.004161,0.032185,0.050143,0.047313,0.102995,0.041285,0.017725,0.414831,0.005867,0.133778,0.838302,0.505911,0.356206,0.336074,0.288888,0.278649,0.283437,0.300305,0.287688,0.296692,0.258531,0.238352,0.194701,0.013138,0.011665,0.032049,0.015464,0.020453,0.012943,0.046397,0.003431,0.004981,0.010818,0.024001,0.005201,0.015056,133.333333,0.132792,128.0,BigRoom
1,1,0.117039,0.108389,3.194001,0.247657,0.250288,1.065668,0.005387,0.199821,-21.775871,1.261364,-0.113015,0.001718,-0.052682,0.204130,0.153013,0.067214,-0.013227,-0.059440,-0.008604,0.114257,0.171009,0.006535,0.002646,0.086485,0.008391,0.016442,0.009006,0.087948,0.002472,0.006549,0.007412,0.015386,0.005978,0.041116,0.043713,0.043721,0.099449,0.039386,0.018946,0.407164,0.003613,0.110334,0.624185,0.476993,0.353151,0.335550,0.283832,0.269621,0.244150,0.246660,0.257190,0.272036,0.269477,0.222393,0.187471,0.006761,0.003152,0.058923,0.009012,0.016106,0.009386,0.071726,0.004461,0.006441,0.007469,0.015499,0.005589,0.019339,120.000000,0.112767,126.0,BigRoom
2,2,0.085308,0.128525,3.123837,0.217205,0.228652,0.789647,0.008247,0.156822,-22.472722,1.425185,0.186749,0.417114,0.076406,0.190803,-0.016302,0.075038,0.107870,0.216874,0.095604,0.020977,-0.037011,0.007143,0.002960,0.220526,0.005639,0.010151,0.007453,0.043907,0.001240,0.004347,0.007989,0.017622,0.002636,0.066049,0.032920,0.037618,0.117704,0.041509,0.022645,0.340130,0.007697,0.085784,1.028740,0.449133,0.297935,0.266731,0.258299,0.275012,0.218368,0.198390,0.210177,0.212533,0.204458,0.197634,0.164910,0.007836,0.003079,0.093865,0.005692,0.008212,0.005451,0.042900,0.001529,0.004556,0.007723,0.017482,0.002901,0.022201,133.333333,0.123373,129.0,BigRoom
3,3,0.103050,0.167042,3.150830,0.233593,0.245032,0.967082,0.006571,0.168083,-21.470751,1.463686,0.226548,0.404531,0.117699,0.081861,0.053974,0.164865,0.014919,0.117090,0.027778,-0.063173,-0.052606,0.010724,0.003340,0.125459,0.005728,0.014695,0.006322,0.072154,0.001628,0.003493,0.011463,0.032204,0.004738,0.046159,0.036349,0.061960,0.134908,0.032564,0.020036,0.365068,0.005215,0.086336,0.769981,0.425496,0.245312,0.260132,0.224220,0.207597,0.199472,0.207818,0.189912,0.185509,0.187273,0.177629,0.164740,0.008330,0.003528,0.061426,0.005443,0.012382,0.004985,0.057999,0.001591,0.003514,0.009477,0.023162,0.004165,0.015379,133.333333,0.158876,129.0,BigRoom
4,4,0.151730,0.148405,3.194498,0.293730,0.267231,1.353005,0.003872,0.292055,-21.371157,1.187854,0.184415,0.363724,0.232119,0.112277,0.107335,0.159296,0.067213,-0.018713,0.091529,0.117344,0.091616,0.009624,0.004031,0.076133,0.008175,0.0

In [5]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- 1-ZCRm: double (nullable = true)
 |-- 2-Energym: double (nullable = true)
 |-- 3-EnergyEntropym: double (nullable = true)
 |-- 4-SpectralCentroidm: double (nullable = true)
 |-- 5-SpectralSpreadm: double (nullable = true)
 |-- 6-SpectralEntropym: double (nullable = true)
 |-- 7-SpectralFluxm: double (nullable = true)
 |-- 8-SpectralRolloffm: double (nullable = true)
 |-- 9-MFCCs1m: double (nullable = true)
 |-- 10-MFCCs2m: double (nullable = true)
 |-- 11-MFCCs3m: double (nullable = true)
 |-- 12-MFCCs4m: double (nullable = true)
 |-- 13-MFCCs5m: double (nullable = true)
 |-- 14-MFCCs6m: double (nullable = true)
 |-- 15-MFCCs7m: double (nullable = true)
 |-- 16-MFCCs8m: double (nullable = true)
 |-- 17-MFCCs9m: double (nullable = true)
 |-- 18-MFCCs10m: double (nullable = true)
 |-- 19-MFCCs11m: double (nullable = true)
 |-- 20-MFCCs12m: double (nullable = true)
 |-- 21-MFCCs13m: double (nullable = true)
 |-- 22-ChromaVector1m: double (null

### Things to do if I have time
- Change column names to remove the annoying numbers ahead of the names.

In [6]:
# we ave 23 classes, with 100 observation per class
#let's make sure of that
df.count()

2300

In [7]:
df.groupBy("class").count().show(25)

+--------------------+-----+
|               class|count|
+--------------------+-----+
|           PsyTrance|  100|
|           HardDance|  100|
|              Breaks|  100|
|  HardcoreHardTechno|  100|
|   IndieDanceNuDisco|  100|
|              Trance|  100|
|           DeepHouse|  100|
|ElectronicaDowntempo|  100|
|           ReggaeDub|  100|
|             Minimal|  100|
|         DrumAndBass|  100|
|             Dubstep|  100|
|             BigRoom|  100|
|              Techno|  100|
|               House|  100|
|         FutureHouse|  100|
|        ElectroHouse|  100|
|           GlitchHop|  100|
|           TechHouse|  100|
|              HipHop|  100|
|           FunkRAndB|  100|
|               Dance|  100|
|    ProgressiveHouse|  100|
+--------------------+-----+



## Preparing 

**Notes:**
- We need to change the class column to zero indexed numeric values as expected for our classification algorithms.
- We have no string features in our data so we will not need to convert any columns or use String Indexer.
- We checked for outliers in the upper and lower 2.5% of the data and decreased the margin of error to 0.05.

In [8]:
# Data Prep function
def MLClassifierDFPrep(df,dependent_var,treat_outliers=True,treat_neg_values=True):

    renamed = df.withColumn("label_str", df[dependent_var].cast(StringType())) #Rename and change to string type
    indexer = StringIndexer(inputCol="label_str", outputCol="label") #Pyspark is expecting the this naming convention 
    indexed = indexer.fit(renamed).transform(renamed)

    numeric_inputs = indexed.columns[1:-3] #to exclude the class, label_str and label columns.
    
    if treat_outliers == True:
        print("We are correcting for non normality now!")
        # empty dictionary d
        d = {}
        # Create a dictionary of quantiles
        for col in numeric_inputs: 
            d[col] = indexed.approxQuantile(col,[0.025,0.975],0.05) #our margin of error is small (0.05) and we checked for outliiers in the upper and lower 2.5%
        #Now fill in the values
        for col in numeric_inputs:
            skew = indexed.agg(skewness(indexed[col])).collect() #check for skewness
            skew = skew[0][0]
            # This function will floor, cap and then log+1 (just in case there are 0 values)
            if skew > 1:
                indexed = indexed.withColumn(col, \
                log(when(indexed[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] ) +1).alias(col))
                print(col+" has been treated for positive (right) skewness. (skew =)",skew,")")
            elif skew < -1:
                indexed = indexed.withColumn(col, \
                exp(when(indexed[col] < d[col][0],d[col][0])\
                .when(indexed[col] > d[col][1], d[col][1])\
                .otherwise(indexed[col] )).alias(col))
                print(col+" has been treated for negative (left) skewness. (skew =",skew,")")

            
    # Produce a warning if there are negative values in the dataframe that Naive Bayes cannot be used. 
    minimums = df.select([min(c).alias(c) for c in df.columns if c in numeric_inputs]) # Calculate the mins for all columns in the df
    min_array = minimums.select(array(numeric_inputs).alias("mins")) # Create an array for all mins and select only the input cols
    df_minimum = min_array.select(array_min(min_array.mins)).collect() # Collect golobal min as Python object
    df_minimum = df_minimum[0][0] # Slice to get the number itself

    features_list = numeric_inputs
    assembler = VectorAssembler(inputCols=features_list,outputCol='features')
    output = assembler.transform(indexed).select('features','label')
    
    # Now check for negative values and ask user if they want to correct that? 
    if df_minimum < 0:
        print(" ")
        print("WARNING: The Naive Bayes Classifier will not be able to process your dataframe as it contains negative values")
        print(" ")
    
    if treat_neg_values == True:
        print("You have opted to correct that by rescaling all your features to a range of 0 to 1")
        print(" ")
        print("We are rescaling you dataframe....")
        scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")

        # Compute summary statistics and generate MinMaxScalerModel
        scalerModel = scaler.fit(output)

        # rescale each feature to range [min, max].
        scaled_data = scalerModel.transform(output)
        final_data = scaled_data.select('label','scaledFeatures')
        final_data = final_data.withColumnRenamed('scaledFeatures','features')
        print("Done!")

    else:
        print("You have opted not to correct that therefore you will not be able to use to Naive Bayes classifier")
        print("We will return the dataframe unscaled.")
        final_data = output
    
    return final_data

In [9]:
dependent_var = 'class'

final_data_b = MLClassifierDFPrep(df,dependent_var)
final_data_b.limit(5).toPandas()

We are correcting for non normality now!
7-SpectralFluxm has been treated for positive (right) skewness. (skew =) 1.6396138160129063 )
22-ChromaVector1m has been treated for positive (right) skewness. (skew =) 2.4162415204309258 )
23-ChromaVector2m has been treated for positive (right) skewness. (skew =) 4.154796693680583 )
24-ChromaVector3m has been treated for positive (right) skewness. (skew =) 1.1974019617504328 )
25-ChromaVector4m has been treated for positive (right) skewness. (skew =) 2.446635863594906 )
26-ChromaVector5m has been treated for positive (right) skewness. (skew =) 2.154482876187508 )
27-ChromaVector6m has been treated for positive (right) skewness. (skew =) 2.01234064472543 )
28-ChromaVector7m has been treated for positive (right) skewness. (skew =) 1.1829228989215521 )
29-ChromaVector8m has been treated for positive (right) skewness. (skew =) 3.7372643733999955 )
30-ChromaVector9m has been treated for positive (right) skewness. (skew =) 2.4117416421548645 )
31-Chr

,label,features
0,0.0,"[0.5198182667002392, 0.30338998025826874, 0.8957733872366486, 0.6189415833674071, 0.6544174970451402, 0.5612689337278991, 0.08812923994167138, 0.5188352540966735, 0.746914974215095, 0.3857754817860692, 0.46627598993117003, 0.4574131487357765, 0.49456194025573946, 0.4468250113272722, 0.7049332862244106, 0.5576767870780662, 0.30533190486406925, 0.2532646840406656, 0.4413994660136605, 0.4728582193292313, 0.4121044353701084, 0.20793124046597242, 0.19119519929695164, 0.0926922452556058, 0.28311745550585105, 0.35550448011364044, 0.27692167409033747, 0.16146999154809194, 0.1032527854426953, 0.1287759691976597, 0.17724081407362677, 0.13031289717079975, 0.14925710276284598, 0.16287297445820995, 0.30805073919433673, 0.28578485268767795, 0.12112376946186845, 0.21774695010946682, 0.1969366603143251, 0.4186178383459218, 0.07572597617577928, 0.43727499019709165, 0.1209945180925387, 0.2576922879146853, 0.252771616497777, 0.35177078832039044, 0.33990198595094473, 0.4286964125667449, 0.5274927099126788, 0.66183116088021, 0.6216328401652649, 0.8056520627267015, 0.6681920548396337, 0.5717806730758072, 0.4579613537623881, 0.24497365723921108, 0.33684035179164684, 0.13745812765393636, 0.24699116194690898, 0.25504485690480133, 0.28615506979065447, 0.22916669993173397, 0.12268948437384192, 0.16646639916919118, 0.1929280704762225, 0.2231349651969688, 0.17501562932120393, 0.27054232841721937, 0.3303063380702889, 0.19118820204898668, 0.5275590551181102]"
1,0.0,"[0.4352954639925659, 0.37399300906442695, 0.8816024910399278, 0.5668259209823236, 0.6616433573297753, 0.5359375521175905, 0.05515668374900831, 0.3983426437163145, 0.8393453537212511, 0.30621107817953064, 0.4209666171591181, 0.31477716279207196, 0.424852237406133, 0.5919133028037521, 0.6999074667323284, 0.49320377681058, 0.4465968792273922, 0.39409662165030984, 0.4691531598017421, 0.6492538625117108, 0.7547449486616228, 0.10720511100389196, 0.054582210199347295, 0.22264268622983951, 0.1574952699943214, 0.20941916030101732, 0.18490692739431783, 0.26859443094410296, 0.08840559158092758, 0.18908607145361794, 0.1270707657504836, 0.06051667172583269, 0.21781634114636605, 0.2560939872534202, 0.26543330359865197, 0.26255806165791395, 0.11437573714962117, 0.20087482431850293, 0.21889405171431306, 0.41036110420646654, 0.04208547580223348, 0.3572144977006486, 0.07029001375188976, 0.22959486941257065, 0.24812619241054468, 0.35064937571713445, 0.32592359381114194, 0.39783976017755784, 0.38169368558490097, 0.44518656916838584, 0.5031518593073012, 0.6908724595507805, 0.7170295603936104, 0.5020229738433585, 0.4217768470206442, 0.107680697868956, 0.08621088325818564, 0.3013758640351787, 0.13705171669090843, 0.18959154348679802, 0.20057751555391018, 0.4307130389400109, 0.16125265659718063, 0.22054186439621873, 0.11842792399880138, 0.12798008494799784, 0.18876583232250915, 0.39221262105798443, 0.2835820867751883, 0.1279369451917282, 0.5118110236220472]"
2,0.0,"[0.2970571291217422, 0.44679648775095127, 0.7434979440255818, 0.4621946471317142, 0.5163865046055455, 0.39252597317186716, 0.11349303239962276, 0.3072243984168306, 0.7713582116745848, 0.3453872740572986, 0.5302432886323369, 0.6070118728960284, 0.5198213134583177, 0.579485204123199, 0.54497627711827, 0.5029477337823209, 0.5735682613480091, 0.7012002411038294, 0.6315038344603556, 0.5138512645698184, 0.37744067642484275, 0.11936443594942055, 0.06162426526785515, 0.5767950161517058, 0.10062326598925089, 0.10952547663006461, 0.14489673249386453, 0.09744274358001824, 0.041724733035999474, 0.11984110951415067, 0.1404531810409523, 0.07516788201480798, 0.0916499735869539, 0.516363061187277, 0.1938947872882848, 0.22309167731391943, 0.14911509038992188, 0.2197379602291736, 0.28545203334647534, 0.33816562661774163, 0.10298155693273585, 0.2733775229205264, 0.16609140799303448, 0.20252595745065832, 0.1641766950909186, 0.20360406803054398, 0.25532236331416835, 0.41626679925933857, 0.2860090765420678, 0.25025276655398676, 0.3205145609262459, 0.4138773

In [10]:
#let's check for a sample
final_data_b.select('features').collect()[1][0]

DenseVector([0.4353, 0.374, 0.8816, 0.5668, 0.6616, 0.5359, 0.0552, 0.3983, 0.8393, 0.3062, 0.421, 0.3148, 0.4249, 0.5919, 0.6999, 0.4932, 0.4466, 0.3941, 0.4692, 0.6493, 0.7547, 0.1072, 0.0546, 0.2226, 0.1575, 0.2094, 0.1849, 0.2686, 0.0884, 0.1891, 0.1271, 0.0605, 0.2178, 0.2561, 0.2654, 0.2626, 0.1144, 0.2009, 0.2189, 0.4104, 0.0421, 0.3572, 0.0703, 0.2296, 0.2481, 0.3506, 0.3259, 0.3978, 0.3817, 0.4452, 0.5032, 0.6909, 0.717, 0.502, 0.4218, 0.1077, 0.0862, 0.3014, 0.1371, 0.1896, 0.2006, 0.4307, 0.1613, 0.2205, 0.1184, 0.128, 0.1888, 0.3922, 0.2836, 0.1279, 0.5118])

## Feature Selection

In [11]:
slicer = VectorSlicer(inputCol="features", outputCol="selected_features", indices=[1,2,3,4,5,6,7,8,9,22,35,36,37,38,39,40,41,42,43,56,69,70])
output = slicer.transform(final_data_b)

In [13]:
final_data = output.select('selected_features','label')
final_data = final_data.withColumnRenamed('selected_features','features')
final_data.limit(2).toPandas()

,features,label
0,"[0.30338998025826874, 0.8957733872366486, 0.6189415833674071, 0.6544174970451402, 0.5612689337278991, 0.08812923994167138, 0.5188352540966735, 0.746914974215095, 0.3857754817860692, 0.19119519929695164, 0.28578485268767795, 0.12112376946186845, 0.21774695010946682, 0.1969366603143251, 0.4186178383459218, 0.07572597617577928, 0.43727499019709165, 0.1209945180925387, 0.2576922879146853, 0.33684035179164684, 0.19118820204898668, 0.5275590551181102]",0.0
1,"[0.37399300906442695, 0.8816024910399278, 0.5668259209823236, 0.6616433573297753, 0.5359375521175905, 0.05515668374900831, 0.3983426437163145, 0.8393453537212511, 0.30621107817953064, 0.054582210199347295, 0.26255806165791395, 0.11437573714962117, 0.20087482431850293, 0.21889405171431306, 0.41036110420646654, 0.04208547580223348, 0.3572144977006486, 0.07029001375188976, 0.22959486941257065, 0.08621088325818564, 0.1279369451917282, 0.5118110236220472]",0.0


In [16]:
#let's check for a sample
final_data.select('features').collect()[1][0]

DenseVector([0.374, 0.8816, 0.5668, 0.6616, 0.5359, 0.0552, 0.3983, 0.8393, 0.3062, 0.0546, 0.2626, 0.1144, 0.2009, 0.2189, 0.4104, 0.0421, 0.3572, 0.0703, 0.2296, 0.0862, 0.1279, 0.5118])

## Training

We have the function that can try many classification algorithms for us by looping through the algorithms we want, we can try:
- one vs rest
- logistic regression
- multi layer preceptron
- linear SVC
- Naive Bayes
- GBT Classifier
- Random Forest
- Decision Tree

**Notes**
- We changed the number of folds for the cross validator to 5 folds for all the algorithms.
- We added regParam for logistic regression.
- We added maxdepth for Random forest and GBT Classifier.

In [14]:
def ClassTrainEval(classifier,features,classes,train,test):

    def FindMtype(classifier):
        # Intstantiate Model
        M = classifier
        # Learn what it is
        Mtype = type(M).__name__
        return Mtype
    
    Mtype = FindMtype(classifier)

    def IntanceFitModel(Mtype,classifier,classes,features,train):
        
        if Mtype == "OneVsRest":
            # instantiate the base classifier.
            lr = LogisticRegression()
            # instantiate the One Vs Rest Classifier
            OVRclassifier = OneVsRest(classifier=lr)
            # Add parameters of your choice here:
            paramGrid = ParamGridBuilder() \
                .addGrid(lr.regParam, [0.1, 0.01]) \
                .build()
            #Cross Validator requires the following parameters:
            crossval = CrossValidator(estimator=OVRclassifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=5)
            # Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
        #----------------------------------------------------------------------------------
        if Mtype == "MultilayerPerceptronClassifier":
            features_count = len(features[0][0])
            layers = [features_count, features_count+1, features_count, classes]
            MPC_classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
            fitModel = MPC_classifier.fit(train)
            return fitModel
        #------------------------------------------------------------------------------------------------
        if Mtype in("LinearSVC","GBTClassifier") and classes != 2: # These classifiers currently only accept binary classification
            print(Mtype," could not be used because PySpark currently only accepts binary classification data for this algorithm")
            return
        #-----------------------------------------------------------------------------------------------
        if Mtype in("LogisticRegression","NaiveBayes","RandomForestClassifier","GBTClassifier","LinearSVC","DecisionTreeClassifier"):
  
            # Add parameters of your choice here:
            if Mtype in("LogisticRegression"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .addGrid(classifier.maxIter, [10, 15,20])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("NaiveBayes"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.smoothing, [0.0, 0.2, 0.4, 0.6]) \
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("RandomForestClassifier"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxDepth, [2, 5, 10])
                             .addGrid(classifier.numTrees, [5, 20, 50])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("GBTClassifier"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxDepth, [2, 5, 10, 20, 30]) \
                             .addGrid(classifier.maxIter, [10, 15,50,100])
                             .build())
                
            # Add parameters of your choice here:
            if Mtype in("LinearSVC"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxIter, [10, 15]) \
                             .addGrid(classifier.regParam, [0.1, 0.01]) \
                             .build())
            
            # Add parameters of your choice here:
            if Mtype in("DecisionTreeClassifier"):
                paramGrid = (ParamGridBuilder() \
                             .addGrid(classifier.maxBins, [10, 20, 40, 80, 100]) \
                             .build())
            
            #Cross Validator requires all of the following parameters:
            crossval = CrossValidator(estimator=classifier,
                                      estimatorParamMaps=paramGrid,
                                      evaluator=MulticlassClassificationEvaluator(),
                                      numFolds=5)
            # Fit Model: Run cross-validation, and choose the best set of parameters.
            fitModel = crossval.fit(train)
            return fitModel
    
    fitModel = IntanceFitModel(Mtype,classifier,classes,features,train)
    
    # Print feature selection metrics
    if fitModel is not None:
        
        if Mtype in("OneVsRest"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype + '\033[0m')
            # Extract list of binary models
            models = BestModel.models
            for model in models:
                print('\033[1m' + 'Intercept: '+ '\033[0m',model.intercept,'\033[1m' + '\nCoefficients:'+ '\033[0m',model.coefficients)

        if Mtype == "MultilayerPerceptronClassifier":
            print("")
            print('\033[1m' + Mtype," Weights"+ '\033[0m')
            print('\033[1m' + "Model Weights: "+ '\033[0m',fitModel.weights.size)
            print("")

        if Mtype in("DecisionTreeClassifier", "GBTClassifier","RandomForestClassifier"):
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Feature Importances"+ '\033[0m')
            print("(Scores add up to 1)")
            print("Lowest score is the least important")
            print(" ")
            featureImportances = BestModel.featureImportances.toArray()
            print(featureImportances)
            
            if Mtype in("DecisionTreeClassifier"):
                global DT_featureImportances
                DT_featureImportances = BestModel.featureImportances.toArray()
                global DT_BestModel
                DT_BestModel = BestModel
            if Mtype in("GBTClassifier"):
                global GBT_featureImportances
                GBT_featureImportances = BestModel.featureImportances.toArray()
                global GBT_BestModel
                GBT_BestModel = BestModel
            if Mtype in("RandomForestClassifier"):
                global RF_featureImportances
                RF_featureImportances = BestModel.featureImportances.toArray()
                global RF_BestModel
                RF_BestModel = BestModel

        if Mtype in("LogisticRegression"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Coefficient Matrix"+ '\033[0m')
            print("You should compares these relative to eachother")
            print("Coefficients: \n" + str(BestModel.coefficientMatrix))
            print("Intercept: " + str(BestModel.interceptVector))
            global LR_coefficients
            LR_coefficients = BestModel.coefficientMatrix.toArray()
            global LR_BestModel
            LR_BestModel = BestModel

        if Mtype in("LinearSVC"):
            # Get Best Model
            BestModel = fitModel.bestModel
            print(" ")
            print('\033[1m' + Mtype," Coefficients"+ '\033[0m')
            print("You should compares these relative to eachother")
            print("Coefficients: \n" + str(BestModel.coefficients))
            global LSVC_coefficients
            LSVC_coefficients = BestModel.coefficients.toArray()
            global LSVC_BestModel
            LSVC_BestModel = BestModel
        
   
    # Set the column names to match the external results dataframe that we will join with later:
    columns = ['Classifier', 'Result']
    
    if Mtype in("LinearSVC","GBTClassifier") and classes != 2:
        Mtype = [Mtype] # make this a list
        score = ["N/A"]
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
    else:
        predictions = fitModel.transform(test)
        MC_evaluator = MulticlassClassificationEvaluator(metricName="accuracy") # redictionCol="prediction",
        accuracy = (MC_evaluator.evaluate(predictions))*100
        Mtype = [Mtype] # make this a string
        score = [str(accuracy)] #make this a string and convert to a list
        result = spark.createDataFrame(zip(Mtype,score), schema=columns)
        result = result.withColumn('Result',result.Result.substr(0, 5))
        
    return result
    #Also returns the fit model important scores or p values

In [15]:
classifiers = [LogisticRegression()
                ,OneVsRest()
               ,LinearSVC()
               ,NaiveBayes()
               ,RandomForestClassifier()
               ,GBTClassifier()
               ,DecisionTreeClassifier()
               ,MultilayerPerceptronClassifier()
              ] 

train,test = final_data.randomSplit([0.7,0.3])
features = final_data.select(['features']).collect()
# Learn how many classes there are in order to specify evaluation type based on binary or multi and turn the df into an object
class_count = final_data.select(countDistinct("label")).collect()
classes = class_count[0][0]

#set up your results table
columns = ['Classifier', 'Result']
vals = [("Place Holder","N/A")]
results = spark.createDataFrame(vals, columns)

for classifier in classifiers:
    new_result = ClassTrainEval(classifier,features,classes,train,test)
    results = results.union(new_result)
results = results.where("Classifier!='Place Holder'")
results.show(100,False)

 
LogisticRegression  Coefficient Matrix
You should compares these relative to eachother
Coefficients: 
DenseMatrix([[ 1.69969071e+00,  1.78714274e+00,  7.04320996e-02,
               2.04497240e+00,  1.05196846e+00, -2.35674909e+00,
              -4.78498930e-01,  3.90248947e+00, -1.06397460e+00,
              -1.24345781e+00,  4.38725675e-01, -4.28675315e-01,
              -2.74419161e+00, -1.47189274e+00, -2.14413027e-01,
              -1.67051138e+00, -1.23242806e+00,  1.37248866e+00,
              -2.65354232e+00,  4.87425971e-01, -2.57564200e-01,
               1.65962773e+00],
             [ 1.33135461e+00,  5.98387116e-01,  7.74861633e-01,
               2.02619108e+00, -1.30959845e+00, -7.51557733e-01,
              -1.18620743e+00, -1.92201229e+00, -2.53643024e+00,
               1.62080189e-01, -1.41701705e+00, -1.32140900e+00,
              -3.17376367e-01, -4.72474380e-01,  1.95462647e+00,
               5.76454462e-01,  1.46866389e+00,  1.62981182e+00,
               3.39

The **Random Forest Classifier** gives the best result so we will continue with it.

In [27]:
def ClassDiag(classifier):
    
    # Fit our model
    C = classifier
    fitModel = C.fit(train)

    # Load the Summary
    trainingSummary = fitModel.summary

    # General Describe
    trainingSummary.predictions.describe().show()

    # View Predictions
    pred_and_labels = fitModel.evaluate(test)
    pred_and_labels.predictions.show()

    # Obtain the objective per iteration
    objectiveHistory = trainingSummary.objectiveHistory
    print(" ")
    print("objectiveHistory:")
    for objective in objectiveHistory:
        print(objective)

    # for multiclass, we can inspect metrics on a per-label basis
    print(" ")
    print("False positive rate by label:")
    for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
        print("label %d: %s" % (i, rate))

    print(" ")
    print("True positive rate by label:")
    for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
        print("label %d: %s" % (i, rate))

    print(" ")
    print("Precision by label:")
    for i, prec in enumerate(trainingSummary.precisionByLabel):
        print("label %d: %s" % (i, prec))

    print(" ")
    print("Recall by label:")
    for i, rec in enumerate(trainingSummary.recallByLabel):
        print("label %d: %s" % (i, rec))

    print(" ")
    print("F-measure by label:")
    for i, f in enumerate(trainingSummary.fMeasureByLabel()):
        print("label %d: %s" % (i, f))

    accuracy = trainingSummary.accuracy
    falsePositiveRate = trainingSummary.weightedFalsePositiveRate
    truePositiveRate = trainingSummary.weightedTruePositiveRate
    fMeasure = trainingSummary.weightedFMeasure()
    precision = trainingSummary.weightedPrecision
    recall = trainingSummary.weightedRecall
    print(" ")
    print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
          % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall)) 

In [28]:
ClassDiag(RandomForestClassifier())

+-------+------------------+------------------+
|summary|             label|        prediction|
+-------+------------------+------------------+
|  count|              1657|              1657|
|   mean|10.980084490042245|10.954133977066988|
| stddev| 6.656983129691139| 6.649548411088633|
|    min|               0.0|               0.0|
|    max|              22.0|              22.0|
+-------+------------------+------------------+

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.03147090008773...| 22.0|[0.91394727347182...|[0.04569736367359...|      22.0|
|[0.05420561519644...|  8.0|[0.22041236735838...|[0.01102061836791...|      20.0|
|[0.05764688971706...| 19.0|[0.88621092354005...|[0.04431054617700...|       1.0|
|[0.07851429939540...| 19.0|[0.20851426675494...|[0.01042571333774...|     

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 36292)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/conda/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/opt/conda/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/conda/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/usr/local/spark/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/usr/local/spark/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/usr/local/spark/python/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.rfile)
  File "/usr/local/spark/python/

Here is the diagnosis of the model on our data, it gives accuracy of 56% with quite low False Positive rate for all labels.<br>
We can further check the false positives and recommend the type of music we got from our model to the people who loves the right type that was supposed to be predicted as we think they are similar and they may like it as well.
